### 二进制数据格式
#### pickle形式
python内置有pickle序列化，pandas对象都有一个将数据以pickle形式保存到磁盘上的to_pickle方法

In [1]:
import pandas as pd
frame = pd.read_csv('input/ex1.csv')

In [2]:
frame

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [3]:
frame.to_pickle('pickle')

In [4]:
pd.read_pickle('pickle')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


> pickle仅建议用于短期存储，因为无法保证今天的pickle数据可以被日后的库读出来

#### HDF5格式
HDF是指层次型数据格式（hierarchical data format）。HDF5是一个流行的工业级库，它是一个C库，具有以下特点：
1. HDF5有很多语言的接口，如JAVA、python和MATLAB等
2. 它能存储多个数据集并支持元数据
3. HDF5支持多种压缩器的即时压缩
4. 高效地分块读写，因此对于非常大的无法直接放入内存的数据集，HDF5是个不错的选择

python为HDF5提供了两个接口：Pytable和h5py
1. h5py提供了一种直接而高级的HDF5 API接口
2. Pytable则抽象了HDF5的许多细节以提供多种灵活的数据容器、表索引、查询功能

虽然可以用PyTables或h5py库直接访问HDF5文件，pandas提供了更为高级的接口，可以简化存储Series和DataFrame对象。HDFStore类可以像字典一样，处理低级的细节：

In [5]:
    import numpy as np
    frame = pd.DataFrame({'a':np.random.randn(100)})

In [6]:
store = pd.HDFStore('mydata.h5')

In [9]:
store['obj1'] = frame

In [11]:
store['obj1_col'] = frame['a']

HDF5文件中的对象可以通过与字典一样的API进行获取：

In [14]:
store['obj1'].head()

,a
0,0.163120
1,-1.160834
2,0.236382
3,2.190185
4,2.090780


HDFStore支持两种存储模式，'fixed'和'table'。后者通常会更慢，但是支持使用特殊语法进行查询操作：

In [15]:
store.put('obj2', frame, format='table')

In [16]:
store.select('obj2', where=['index>10 and index<=15'])

,a
11,1.094710
12,-2.447485
13,1.250322
14,0.363331
15,0.872663


In [17]:
store.close()

put是store['obj2'] = frame方法的显示版本，允许我们设置其它的选项，比如格式。

pandas.read_hdf函数可以快捷使用这些工具：

In [18]:
frame.to_hdf('mydata.h5','obj3', format='table')

In [19]:
pd.read_hdf('mydata.h5','obj3',where=['index<10'])

,a
0,0.163120
1,-1.160834
2,0.236382
3,2.190185
4,2.090780
5,-0.844220
6,-0.832506
7,0.191474
8,-0.668625
9,0.437684


> 注意：HDF5不是数据库。它最适合用作“一次写多次读”的数据集。虽然数据可以在任何时候被添加到文件中，但如果同时发生多个写操作，文件就可能会被破坏。

#### Excel格式文件
ExcelFile类用到了xlrd和openpyxl包，先安装它们

**读取Excel**

1、创建ExcelFile实例

In [6]:
xlsx = pd.ExcelFile('input/ex1.xlsx')

2、读取DataFrame

In [9]:
pd.read_excel(xlsx, 'Sheet1')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


如果要读取一个文件中的多个表单，创建ExcelFile会更快，但你也可以将文件名传递到pandas.read_excel：

In [12]:
pd.read_excel('input/ex1.xlsx', 'Sheet1')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


**写入Excel**

1、生成ExcelWriter

In [14]:
writer = pd.ExcelWriter('ex2.xlsx')

2、to_excel方法写入数据

In [15]:
frame.to_excel(writer, 'Sheet1')

In [16]:
writer.save()

也可以直接将路径传入to_excel方法中

In [17]:
frame.to_excel('ex2.xlsx')